# Chapter 0 - Operating system interfaces

The job of an **operating system** is:
* to share a computer among multiple programs.
* to provide a more useful set of services than the hardware alone supports through interface.

**Process** is a running program, it has **memory** to contian:
* **instructions**: implement the program’s computation.
* **data**: the variables on which the computation acts.
* **stack**: organizes the program’s procedure calls.


When a process needs to invoke a kernel service, it invokes a procedure call in the operating system interface, the procedure is called a **system call**. The system call enters the kernel, the kernel performs the service and returns. Thus a process alternates between executing in **user space** and **kernel space**.

The kernel uses the CPU’s **hardware protection mechanisms** to ensure that each process executing in user space can access only its own memory. The kernel executes with the hardware privileges required to implement these protections; user programs execute without those privileges. When a user program invokes a system call, the hardware raises the privilege level and starts executing a pre-arranged function in the kernel.


![Figure0-1](./images/figure0-1.png)


The **shell** is an ordinary program that reads commands from the user and executes them. The fact that the shell is a user program, not part of the kernel, illustrates the power of the system call interface.

## Processes and memory

An xv6 process consists of user-space memory (instructions, data, and stack) and per-process state private to the kernel.

**Time-share processes**: it transparently switches the available CPUs among the set of processes waiting to execute.

The **fork** system call: Fork creates a new process, called the **child process**, with exactly the same memory contents as the calling process, called the **parent process**. Fork returns in both the parent and the child, in the parent, fork returns the child’s pid; in the child, it returns zero. Although the child has the same memory contents as the parent initially, the parent and child are executing with different memory and different registers: changing a variable in one does not affect the other.

![Figure0-2](./images/figure0-2.png)

The **exec** system call replaces the calling process’s memory with a new memory image loaded from a file stored in the file system. While **fork** allocates the memory required for the child’s copy of the parent’s memory, and **exec** allocates enough memory to hold the executable file.

A process that needs more memory at run-time (perhaps for malloc) can call sbrk(n) to grow its data memory by n bytes; sbrk returns the location of the new memory.

## I/O and File descriptors

A **file descriptor** is a small integer representing a kernel-managed object that a process may read from or write to.

A process:
* reads from file descriptor 0 (standard input).
* writes output to file descriptor 1 (standard output).
* writes error messages to file descriptor 2 (standard error).

The **read** and **write** system calls read bytes from and write bytes to open files named by file descriptors.

The **close** system call releases a file descriptor, making it free for reuse by a future open, pipe, or dup system call.

The **open** system call uses the smallest available file descriptor for the newly opened file.

File descriptors and fork interact to make I/O redirection easy to implement, `cat < input.txt`:

``` C
char *argv[2];
     argv[0] = "cat";
     argv[1] = 0;
     if(fork() == 0) {
       close(0);
       open("input.txt", O_RDONLY);
       exec("cat", argv);
}
```


Although fork copies the file descriptor table, each underlying file offset is shared between parent and child:

``` C
if(fork() == 0) {
       write(1, "hello ", 6);
       exit();
     } else {
       wait();
       write(1, "world\n", 6);
     }
}

Output >>> hello world
```

The **dup** system call duplicates an existing file descriptor, returning a new one that refers to the same underlying I/O object. The `2>&1` tells the shell to give the command a file descriptor 2 that is a duplicate of descriptor 1.

File descriptors are a powerful abstraction, because they hide the details of what pipe they are connected to: a process writing to file descriptor 1 may be writing to a file, to
a device like the console, or to a pipe.

## Pipes

A **pipe** is a small kernel buffer exposed to processes as a pair of file descriptors, one for reading and one for writing. Writing data to one end of the pipe makes that data available for reading from the other end of the pipe. Pipes provide a way for processes to communicate.

Four advantages over temporary files:

* pipes automatically clean themselves up.
* pipes can pass arbitrarily long streams of data, while file redirection requires enough free space on disk to store all the data.
* pipes allow for parallel execution of pipeline stages, while the file approach requires the first program to finish before the second starts.
* pipes’ blocking reads and writes are more efficient than the non-blocking semantics of files.

## File system

**Data files** are uninterpreted byte arrays, **directories** contain named references to data files and other directories. The directories form a tree, starting at a special directory called the root.


The **mkdir** systmem call creates a new directory.

The **open** system call with the O_CREATE flag creates a new data file.

The **mknod** system call creates a new device file.

The **fstat** retrieves information about the object a file descriptor refers to:

``` C
#define T_DIR  1 // Directory
#define T_FILE 2 // File
#define T_DEV  3 // Device

struct stat {
    short type;  // Type of file
    int dev;     // File system’s disk device
    uint ino;    // Inode number
    short nlink; // Number of links to file
    uint size;   // Size of file in bytes
};
```

Each **inode** is identified by a unique inode number.


The **link** system call creates another file system name referring to the same inode as an existing file.

The **unlink** system call removes a name from the file system. The file’s inode and the disk space holding its content are only freed when the file’s link count is zero and no file descriptors refer to it